Welcome to day 1 of the 5-Day Data Challenge! Today, we're going to be looking at how to deal with missing values. To get started, click the blue "Fork Notebook" button in the upper, right hand corner. This will create a private copy of this notebook that you can edit and play with. Once you're finished with the exercises, you can choose to make your notebook public to share with others. :)

> **Your turn!** As we work through this notebook, you'll see some notebook cells (a block of either code or text) that has "Your Turn!" written in it. These are exercises for you to do to help cement your understanding of the concepts we're talking about. Once you've written the code to answer a specific question, you can run the code by clicking inside the cell (box with code in it) with the code you want to run and then hit CTRL + ENTER (CMD + ENTER on a Mac). You can also click in a cell and then click on the right "play" arrow to the left of the code. If you want to run all the code in your notebook, you can use the double, "fast forward" arrows at the bottom of the notebook editor.

Here's what we're going to do today:

* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)

Let's get started!

# Take a first look at the data
________

The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

In [1]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
nfl_data = pd.read_csv("../input/nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv")
sf_permits = pd.read_csv("../input/building-permit-applications-data/Building_Permits.csv")

# set seed for reproducibility
np.random.seed(0) 

The first thing I do when I get a new dataset is take a look at some of it. This lets me see that it all read in correctly and get an idea of what's going on with the data. In this case, I'm looking to see if I see any missing values, which will be reprsented with `NaN` or `None`.

In [2]:
# look at a few rows of the nfl_data file. I can see a handful of missing data already!
nfl_data.sample(5)

Yep, it looks like there's some missing values. What about in the sf_permits dataset?

In [3]:
# your turn! Look at a couple of rows from the sf_permits dataset. Do you notice any missing data?

sf_permits.sample(5)

# See how many missing data points we have
___

Ok, now we know that we do have some missing values. Let's see how many we have in each column. 

In [4]:
# get the number of missing data points per column
missing_values_count = nfl_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

In [5]:
# how many total missing values do we have?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

In [6]:
# your turn! Find out what percent of the sf_permit dataset is missing
# get the number of missing data points per column
missing_values_count_sf_permits = sf_permits.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count_sf_permits[0:10]

There are several columns with missing values.

In [7]:
# how many total missing values do we have?
total_cells_sf_permits = np.product(sf_permits.shape)
total_missing_sf_permits = missing_values_count_sf_permits.sum()

# percent of data that is missing
(total_missing_sf_permits/total_cells_sf_permits) * 100

~26% of the cells have missing data.

# Figure out why the data is missing
____
 
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it: 

In [8]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Your turn!

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

In [19]:
print(sf_permits.shape);
print(sf_permits["Street Number Suffix"])

Not all addresses have a street number suffix, so the nulls are probably missing because they don't exist. The missing zipcode values were probably not recorded, since all addresses should have a zipcode.

# Drop missing values
___

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [20]:
# remove all the rows that contain a missing value
nfl_data.dropna()

Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

In [22]:
# remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

In [23]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

In [24]:
# Your turn! Try removing all the rows from the sf_permits dataset that contain missing values. How many are left?
# remove all columns with at least one missing value
sf_permits.dropna()

In [25]:
# Now try removing all the columns with empty values. Now how much of your data is left?
columns_with_na_dropped_sf_permits = sf_permits.dropna(axis=1)
columns_with_na_dropped_sf_permits.head()

In [26]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % sf_permits.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped_sf_permits.shape[1])

# Filling in missing values automatically
_____

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [27]:
# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [28]:
# replace all NA's with 0
subset_nfl_data.fillna(0)

I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [29]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
subset_nfl_data.fillna(method = 'bfill', axis=0).fillna("0")

Filling in missing values is also known as "imputation", and you can find more exercises on it [in this lesson, also linked under the "More practice!" section](https://www.kaggle.com/dansbecker/handling-missing-values). First, however, why don't you try replacing some of the missing values in the sf_permit dataset?

# Imputation of sf_permits dataset
To start, let's take a look at the columns with missing values in the sf_permits dataset. 

In [30]:
sf_permits[sf_permits.columns[sf_permits.isnull().any()].tolist()].head()

So there are 31 columns with missing data. Let's take a look at the data types of these columns.

In [31]:
sf_permits[sf_permits.columns[sf_permits.isnull().any()].tolist()].dtypes

The first thing that comes to my attention is that there are only two data types in the missing data columns, object and float64. However, there are missing date values that need to be filled.  Let's get the names of all of the date columns and save them to a list.

In [32]:
date_columns = [col for col in sf_permits.columns.tolist() if col.endswith('Date')]

In order to fill in the missing date value we'll need to convert the columns to the correct data type.

In [33]:
for col in date_columns:
    sf_permits[col] = pd.to_datetime(sf_permits[col], format='%m/%d/%Y')

Let's verify that the date columns have the appropriate data types after the conversion.

In [34]:
sf_permits[date_columns].dtypes

Great! Now we're ready to fill in missing values in the date columns. Let's take a look at the summary statistics for these columns.

In [35]:
sf_permits_dates = sf_permits[date_columns]
sf_permits_dates.describe()

The first and last date values are within the years 20(12,13) and 2018 respectively for all date columns except Permit Expiration Date which has a last date value with year 2024. We should take note of this in case we need to use an average value during imputation. 
Let's check which of the date columns have missing values.

In [36]:
sf_permits_dates.columns[sf_permits_dates.isnull().any()].tolist()

So Permit Creation Date, Current Status Date, and Filed Date have no missing values while Issued Date,Completed Date,First Construction Document Date, and Permit Expiration Date do have missing values. Perhaps there is a pattern between these various date columns. It looks like Filed Date and Issued Date are repeated in several rows. However, we should verify this assumption before performing the imputation. We should also get the total row count of the data frame since it will be needed to compute ratios. 

In [37]:
total_row_count = sf_permits.shape[0]
total_row_count

So there are ~199k rows in total.
Now let's get the count of rows such that Filed Date and Issued Date are the same.

In [38]:
filed_issued_dates_row_count_matching = sf_permits[sf_permits['Filed Date'] == sf_permits['Issued Date']].shape[0]
filed_issued_dates_row_count_matching

In [39]:
filed_issued_dates_row_count_matching/total_row_count


So there are roughly 58% of rows such that filed date and issued date are the same. To be safe, let's get the average difference between these two date columns. We'll use the average of the difference to fill in missing Issued Date values.

In [40]:
average_days_between_filed_and_issued = abs((sf_permits['Filed Date'] - sf_permits['Issued Date']).mean().days)
average_days_between_filed_and_issued

So the average is 27 days. Let's fill in missing File Date values by taking Filed Date and adding the average days between Filed Date and Issued Date.

In [41]:
sf_permits['Issued Date'] = sf_permits['Issued Date'].fillna(sf_permits['Filed Date'] +  pd.DateOffset(average_days_between_filed_and_issued))

Great, we've filled in missing Issued Date Values. Now let's perform imputation on the Completed Date column. Let's see how many dates match between Issued Date and Completed Date.

In [42]:
issued_completed_dates_row_count_matching = sf_permits[sf_permits['Issued Date'] == sf_permits['Completed Date']].shape[0]
issued_completed_dates_row_count_matching

Well, 323 is a pretty low number. Let's get the average duration between Issued Date and Completed Date. 

In [43]:
issued_to_completed_duration_average = abs((sf_permits['Issued Date'] - sf_permits['Completed Date']).mean().days)
issued_to_completed_duration_average

The average number of days is 164. Like before, let's use the average to fill in the missing Completed Date values

In [44]:
sf_permits['Completed Date'] = sf_permits['Completed Date'].fillna(sf_permits['Issued Date'] +  pd.DateOffset(issued_to_completed_duration_average))

Now let's see if there is a pattern between Issued Date and First Construction Document Date.

In [45]:
issued_first_construction_row_count_matching = sf_permits[sf_permits['Issued Date'] == sf_permits['First Construction Document Date']].shape[0]
issued_first_construction_row_count_matching

In [46]:
issued_first_construction_row_count_matching/sf_permits.shape[0]

~91% of of the rows in our subset dataframe have matching Issued Date and First Construction Document Date values. Let's go ahead and fill in missing First Construction Document Date values with corresponding Issued Date values.

In [47]:
sf_permits['First Construction Document Date'] = sf_permits['First Construction Document Date'].fillna(sf_permits['Issued Date'])

Just one date column left. Let's see what the average duration in days is between Issued Date and Permit Expiration Date. Since the first and last dates in this column have a greater range, with last falling into the year 2024, we can use the median value to avoid outliers.

In [48]:
issued_to_permit_expiration_duration_average = abs(sf_permits['Permit Expiration Date'] - sf_permits['Permit Creation Date']).median()
issued_to_permit_expiration_duration_average.days

Hmmm...so the average is 360 days and there are 365 days in a year.  Let's just use 365 days since that sounds like a reason duration for a permit.

In [49]:
sf_permits['Permit Expiration Date'] = sf_permits['Permit Expiration Date'].fillna(sf_permits['Filed Date'] +  pd.DateOffset(365))

Great! We're done with performing imputations on the date columns. We still have several columns with missing values. For columns with numeric data types we'll replace all NA's with the value that comes directly after it in the same column and then replace all the reamining na's with 0. For columns with categorical data types we'll replace all NA's with the value that comes directly after it in the same column and then replace all the reamining na's with Unknown.  Let's make sure that the columns have the appropriate data types.

In [50]:
sf_permits.dtypes

'Zipcode', 'Supervisor District', and 'Street Number' are categorical attributes. 'Record ID' is a unique identifier. Unit is a bit ambiguous, it would be the number of units or the respective unit number of the property. We'll just leave its data type alone. Let's convert these columns to objects. 

In [51]:
for col in ['Zipcode', 'Record ID', 'Supervisor District', 'Street Number']:
    sf_permits[col] = sf_permits[col].astype('O')

We should also exclude columns that end in 'Type', since that's an indicator that it's a categorical attribute.

In [52]:
for col in [col for col in sf_permits.columns.tolist() if col.endswith('Type')]:
     sf_permits[col] = sf_permits[col].astype('O')

Let's fill in missing values in numeric columns using the pandas select_dtypes function.

In [53]:
numeric_columns = [col for col in sf_permits.select_dtypes(exclude = ['datetime64[ns]', 'object'])]

In [54]:
sf_permits[numeric_columns] = sf_permits[numeric_columns].fillna(method = 'bfill', axis=0).fillna("0")

Let's fill in missing categorical values. 

In [55]:
categorical_columns = [col for col in sf_permits.select_dtypes(include='object')]
sf_permits[categorical_columns] = sf_permits[categorical_columns].fillna(method = 'bfill', axis=0).fillna("Unknown")

Finally, we should make sure there are no missing values left in the dataframe.

In [56]:
assert len(sf_permits.columns[sf_permits.isnull().any()]) == 0, 'There are still missing values in the dataset!'

And that's it for today! If you have any questions, be sure to post them in the comments below or [on the forums](https://www.kaggle.com/questions-and-answers). 

Remember that your notebook is private by default, and in order to share it with other people or ask for help with it, you'll need to make it public. First, you'll need to save a version of your notebook that shows your current work by hitting the "Commit & Run" button. (Your work is saved automatically, but versioning your work lets you go back and look at what it was like at the point you saved it. It also let's you share a nice compiled notebook instead of just the raw code.) Then, once your notebook is finished running, you can go to the Settings tab in the panel to the left (you may have to expand it by hitting the [<] button next to the "Commit & Run" button) and setting the "Visibility" dropdown to "Public".

# More practice!
___

If you're looking for more practice handling missing values, check out these extra-credit\* exercises:

* [Handling Missing Values](https://www.kaggle.com/dansbecker/handling-missing-values): In this notebook Dan shows you several approaches to imputing missing data using scikit-learn's imputer. 
* Look back at the `Zipcode` column in the `sf_permits` dataset, which has some missing values. How would you go about figuring out what the actual zipcode of each address should be? (You might try using another dataset, like the [OpenAddresses dataset](https://www.kaggle.com/openaddresses/openaddresses-us-west).) 

\* no actual credit is given for completing the challenge, you just learn how to clean data real good :P